# Lab 8: Training Deep Recurrent Neural Network - Part 2

- นาย นันท์มนัส ตั้งประเสริฐ, 63070501040
- นาย สัณหณัฐ พรมจรรย์, 63070501069

**Note: Please name your file**

## Lab Instruction - Language Modelling and Text Classification

In this lab, you will learn to train a deep recurrent neural network using LSTM with the Keras library using the Tensorflow backend. Your task is to implement the natural language modelling and text generation.

Select your favourite book from https://www.gutenberg.org/browse/scores/top and download it as a text file. Then, you will train your language model using RNN-LSTM.

- Language model (in Thai): http://bit.ly/language_model_1
- Tutorial on how to create a language model (in English): https://medium.com/@shivambansal36/language-modelling-text-generation-using-lstms-deep-learning-for-nlp-ed36b224b275

To evaluate the model, the perplexity measurement is used: https://stats.stackexchange.com/questions/10302/what-is-perplexity

Last, fine-tune your model. You have to try different hyperparameter or adding more data. Discuss your result.



**The total lab score is 15 which will be evaluated as follows:**</br>
1. Specification (Do as the instruction said. This include the model tuning section where you have to do a proper amount of tuning) - 7 points
2. Design of logic (No weired things in the process) -  4points
3. Journaling (Communicate your thought process, comment your code, and discuss result & analyse **in every step**) - 4 points



#### 1. Load your data

In [ ]:
# Import require library
from keras import *
from keras.preprocessing import text
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split

import _utils as fn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras

In [ ]:
# Load data
import csv

# Load data
file = open("/content/the_fall_of_the_house_of_usher.txt","r",encoding="utf8", errors='ignore')
raw_text = file.read()

In [ ]:
raw_text[:200]

"\ufeffThe Fall of the House of Usher\n\n\n  Son coeur est un luth suspendu;\n  Sitot qu'on le touche il resonne.\n    DE BERANGER.\n\n\n\nDuring the whole of a dull, dark, and soundless day in the\nautumn of the yea"

In [ ]:
chars = sorted(list(set(raw_text)))

In [ ]:
print("Total characters: ", len(chars))
print("Total word: ", len(raw_text.split()))

Total characters:  86
Total word:  9990


#### 2. Data Preprocessing

*Note that only story will be used as a dataset, footnote and creddit are not include.*

The symbol '\n' is indicated the end of the line ``<EOS>``, which is for our model to end the sentence here.

To create a corpus for your model. The following code is can be used:</br>
*Note that other techniques can be used*

```python
# cut the text in semi-redundant sequences of maxlen characters.
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
```

The code loop through the data from first word to the last word. The maxlen define a next n word for a model to predict.


In [ ]:
from keras.preprocessing.sequence import pad_sequences

In [ ]:
# Adding end of string symbol use .replace   to replace data_text with  [  \n\n', " <EOS> " ]
raw_text = raw_text.replace('\n\n', " <EOS> ")
raw_text[:200]


"\ufeffThe Fall of the House of Usher <EOS> \n  Son coeur est un luth suspendu;\n  Sitot qu'on le touche il resonne.\n    DE BERANGER. <EOS>  <EOS> During the whole of a dull, dark, and soundless day in the\nau"

In [ ]:
# Preprocessing
# Create corpus & Vectorization

#Preprocessing
# Create corpus & Vectorization

tokenizer = text.Tokenizer()

# basic cleanup
corpus = raw_text.lower().split("\n")

# tokenization
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# pad sequences
max_sequence_len = max([len(x) for x in input_sequences])

# Pre padding
input_sequences = np.array(sequence.pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

# One-hot label
label = keras.utils.to_categorical(label, num_classes=total_words)

In [ ]:
print('Max sequence len: %s' % max_sequence_len)
print('Total word len: %s' % total_words)

Max sequence len: 35
Total word len: 2548


In [ ]:
n_gram_sequence[0]

2546

In [ ]:
print(predictors[0])

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 985]


In [ ]:
print(label[0])

[0. 0. 0. ... 0. 0. 0.]


#### 3. Language Model

Define RNN model using LSTM and word embedding representation</br>
We will used perplexity as a metrics

```python
def perplexity(y_true, y_pred):
    cross_entropy = keras.backend.categorical_crossentropy(y_true, y_pred)
    perplexity = keras.backend.pow(2.0, cross_entropy)
    return perplexity
```

To used custom metrics function > https://keras.io/metrics/

For a loss function `categorical_crossentropy` is used, any optimzation method can be applied.


In this lab, we will used perplexity as a metrics


```
def perplexity(y_true, y_pred):
    cross_entropy = keras.backend.categorical_crossentropy(y_true, y_pred)
    perplexity = keras.backend.pow(2.0, cross_entropy)
    return perplexity
```


To used custom metrics function > https://keras.io/metrics/

In [ ]:
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
import keras.backend

In [ ]:
def perplexity(y_true, y_pred):
    cross_entropy = keras.backend.categorical_crossentropy(y_true, y_pred)
    perplexity = keras.backend.pow(2.0, cross_entropy)
    return perplexity

In [ ]:
# Define your model
# Used Word Embedding

m_1 = models.Sequential()
m_1.add(layers.Embedding(total_words, 512,input_length=max_sequence_len-1,name='Embedding'))
m_1.add(layers.LSTM(512, kernel_initializer = 'he_normal',
                      dropout=0.5,
                      return_sequences=True,
                     name='LSTM1'))
m_1.add(layers.LSTM(256, kernel_initializer = 'he_normal',
                     dropout=0.5,
                     name='LSTM2'))
m_1.add(layers.Dense(total_words, activation='softmax',name='Output'))

m_1.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=[perplexity])
m_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Embedding (Embedding)       (None, 34, 512)           1304576   
                                                                 
 LSTM1 (LSTM)                (None, 34, 512)           2099200   
                                                                 
 LSTM2 (LSTM)                (None, 256)               787456    
                                                                 
 Output (Dense)              (None, 2548)              654836    
                                                                 
Total params: 4846068 (18.49 MB)
Trainable params: 4846068 (18.49 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='loss', verbose=1)

In [ ]:
history = m_1.fit(predictors, label,batch_size=32, epochs=100, callbacks=[early_stop])

Epoch 1/100
295/295 [==============================] - 25s 45ms/step - loss: 6.4196 - perplexity: 205.4448
Epoch 2/100
295/295 [==============================] - 5s 16ms/step - loss: 6.2510 - perplexity: 188.0915
Epoch 3/100
295/295 [==============================] - 7s 22ms/step - loss: 6.1514 - perplexity: 185.3671
Epoch 4/100
295/295 [==============================] - 5s 16ms/step - loss: 6.0648 - perplexity: 180.6036
Epoch 5/100
295/295 [==============================] - 4s 13ms/step - loss: 5.9727 - perplexity: 174.1084
Epoch 6/100
295/295 [==============================] - 4s 13ms/step - loss: 5.8746 - perplexity: 164.0115
Epoch 7/100
295/295 [==============================] - 4s 15ms/step - loss: 5.7946 - perplexity: 156.7630
Epoch 8/100
295/295 [==============================] - 4s 14ms/step - loss: 5.7318 - perplexity: 152.5755
Epoch 9/100
295/295 [==============================] - 4s 13ms/step - loss: 5.6821 - perplexity: 148.5667
Epoch 10/100
295/295 [=======================

#### 4. Evaluate your model

In [ ]:
# Create a function to evaluate your model using perplexity measurment (You can try adding other measurements as well)
def evaluate_result(features, label, model ):
    model.evaluate(features, label)

In [ ]:
evaluate_result(predictors, label, m_1)

295/295 [==============================] - 3s 7ms/step - loss: 0.5813 - perplexity: 3.0309


#### 5. Text generating

In [ ]:
def generate_text(seedtext, next_words, max_sequence_len, model):
  for j in range(next_words):
    token_list = tokenizer.texts_to_sequences([seedtext])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    #predicted = model.predict_classes(token_list, verbose=0)
    predict_x=model.predict(token_list)
    predicted =np.argmax(predict_x,axis=1)

    output_word = ""
    for word, index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
    seedtext +=" " + output_word
  return seedtext

In [ ]:
# generate your sample text

seed_text = input('Enter your start sentence:')
#generate_text , Input , Num_next_word,Max_sequence,Model
gen_text = generate_text(seed_text,10,max_sequence_len,m_1)

Enter your start sentence:I had
1/1 [==============================] - 0s 20ms/step


In [ ]:
gen_text

'I had knew up for the pressure of the house and the'

### **Try out different hyperparameter & model architecture**


In [ ]:
# Define your model
# Used Word Embedding

m_2 = models.Sequential()
m_2.add(layers.Embedding(total_words, 256, input_length=max_sequence_len-1, name='Embedding'))
m_2.add(layers.LSTM(512, kernel_initializer='he_normal',
                   dropout=0.2,
                   recurrent_dropout=0.2,
                   return_sequences=True,
                   name='LSTM1'))
m_2.add(layers.LSTM(256, kernel_initializer='he_normal',
                   dropout=0.2,
                   recurrent_dropout=0.2,
                   name='LSTM2'))
m_2.add(layers.Dense(total_words, activation='softmax', name='Output'))

m_2.compile(optimizer='adam',loss='categorical_crossentropy', metrics=[perplexity])
m_2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Embedding (Embedding)       (None, 34, 256)           652288    
                                                                 
 LSTM1 (LSTM)                (None, 34, 512)           1574912   
                                                                 
 LSTM2 (LSTM)                (None, 256)               787456    
                                                                 
 Output (Dense)              (None, 2548)              654836    
                                                                 
Total params: 3669492 (14.00 MB)
Trainable params: 3669492 (14.00 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
history = m_2.fit(predictors, label,batch_size=32, epochs=100, callbacks=[early_stop])

Epoch 1/100
295/295 [==============================] - 110s 349ms/step - loss: 6.5204 - perplexity: 285.4857
Epoch 2/100
295/295 [==============================] - 91s 310ms/step - loss: 6.0220 - perplexity: 199.1685
Epoch 3/100
295/295 [==============================] - 90s 306ms/step - loss: 5.7777 - perplexity: 160.8929
Epoch 4/100
295/295 [==============================] - 89s 303ms/step - loss: 5.5213 - perplexity: 134.1877
Epoch 5/100
295/295 [==============================] - 90s 305ms/step - loss: 5.2743 - perplexity: 106.8913
Epoch 6/100
295/295 [==============================] - 89s 302ms/step - loss: 5.0565 - perplexity: 89.0026
Epoch 7/100
295/295 [==============================] - 91s 308ms/step - loss: 4.8409 - perplexity: 72.6490
Epoch 8/100
295/295 [==============================] - 90s 304ms/step - loss: 4.6942 - perplexity: 74.5351
Epoch 9/100
295/295 [==============================] - 89s 302ms/step - loss: 4.4718 - perplexity: 51.7844
Epoch 10/100
295/295 [=========

In [ ]:
evaluate_result(predictors, label, m_2)

295/295 [==============================] - 8s 25ms/step - loss: 0.1376 - perplexity: 1.2613


In [ ]:
# generate your sample text

seed_text = input('Enter your start sentence:')
#generate_text , Input , Num_next_word,Max_sequence,Model
gen_text = generate_text(seed_text,10,max_sequence_len,m_2)

Enter your start sentence:I had
1/1 [==============================] - 0s 37ms/step


In [ ]:
gen_text

'I had been led to his resolution so he told me by'

# ***Answers***

**1. ลองใช้modelแบบที่เตรียมมาให้ คือ drop out rate ที่0.5 แต่เพิ่ม  return_sequence=true ซึ่งจะทำให้ส่งผ่านคำตอบจาก time stepเก่ามายัง time step ใหม่ได้ดีกว่าปกติ ซึ่งนิยมใช้กับ LSTM epoch เป็น100และใส่ earlystopเพื่อกันการoverfit จึงได้มีการรันไปถึง100 epoch โดยไม่มีการหยุดก่อน แต่ก็ยังมี perplexity ที่สูงถึง 3.03**

**2. ลองปรับmodelดังนี้**
- ลด dimension จาก 512 เป็น 216 ช่วยลดความซับซ้อนและเวลาในการเทรน แต่ก็ยังทำให้
เก็บข้อมูลที่สำคัญเอาไว้ได้ ในขณะที่การมี embed dimension มากเกินไปก็อาจทำให้overfitได้
- dropout ลดเหลือ0.2 เนื่องจากdataมีขนาดเล็กจึงคิดว่า dropout 0.5ตอนแรกนั้นมากเกินไป
อาจนำสู่การสูญเสียข้อมูลที่สำคัญได้
- recurrent dropout ใส่ไว้ที่ 0.2 ซึ่งจะทำให้มีการ drop outในทุกๆ time step แทน
train stepเหมือน dropoutทั่วๆไป ซึ่งเป็นวิธีที่มักใช้กันใน LSTMS , GRU
- ลองเปลี่ยน optimizer เป็น adam ที่เป็นตัวยอดนิยม แต่ก็ใช่ว่าการใช้ adam จะดีกว่าเสมอไปจึงต้องทำการทดลองแล้วผลลัพทธ์ที่ได้คือดีกว่า


**โมเดลที่ 2 ได้ preplexity ที่ดีกว่าและเมื่อทำการ generate ประโยคก็ได้ประโยคที่ดูถูกต้องและมีความเป็นมนุษย์มากกว่าด้วย**

####[Speacial] 7. Help your model to generate a short story

**Example** https://medium.com/deep-writing/harry-potter-written-by-artificial-intelligence-8a9431803da6

Write your result in a `markdown` cell

In [ ]:
# Create your short-story from your model (Add your creativity here)

# Set the initial seed texts for each chapter
seed_texts = ["In the greenest of our valleys", "Banners yellow, glorious, golden, On its roof did float and flow", "Wanderers in that happy valley", "All with pearl and ruby glowing was the fair palace door", "But evil things, in robes of sorrow"]
chapter_count = 1
chapter_length = 200
generated_story = ""

# Generate the story in chapters
for seed_text in seed_texts:
    chapter = generate_text(seed_text, chapter_length, max_sequence_len, m_1)
    generated_story += "\n\nChapter " + str(chapter_count) + "\n\n" + chapter
    chapter_count += 1

1/1 [==============================] - 0s 21ms/step


In [ ]:
# Display the generated story
print(generated_story)



Chapter 1

In the greenest of our valleys my sufficient if i perceive my mighty would fro alone i was tell my features to the ordinary floor which i have was my feet with the manner it of the ordinary did now lifted now lifted his lifted to mad feet and if i was it was i was heard it he in a small fancy at as it although by where or through the laws of he heart usher did by u s it law the law the door was the hours which now agitated friend the door in the entering gust its lifted to her mace ordinary did with the terrible of the the ordinary of the felt and the ordinary ethelred now lifted upon the at of the sound it was upon it removed i immediately elapsed and and struck and an and struck and now her lady upon the coffin and the now might now fro to the like of the chamber of the of the heard the now of the spoke of the of the was of the of her spoke of her lips of it to her features of her like her spoke of and grief her of a has at at her

Chapter 2

Banners yellow, glorious, go

Chapter 1

In the greenest of our valleys my sufficient if i perceive my mighty would fro alone i was tell my features to the ordinary floor which i have was my feet with the manner it of the ordinary did now lifted now lifted his lifted to mad feet and if i was it was i was heard it he in a small fancy at as it although by where or through the laws of he heart usher did by u s it law the law the door was the hours which now agitated friend the door in the entering gust its lifted to her mace ordinary did with the terrible of the the ordinary of the felt and the ordinary ethelred now lifted upon the at of the sound it was upon it removed i immediately elapsed and and struck and an and struck and now her lady upon the coffin and the now might now fro to the like of the chamber of the of the heard the now of the spoke of the of the was of the of her spoke of her lips of it to her features of her like her spoke of and grief her of a has at at her

Chapter 2

Banners yellow, glorious, golden, On its roof did float and flow with a feeling of of of volunteers and myself became passed myself the us who who us like or usher an bounden upon the terrible feeling the he hung did i was heard was as his features of the visit floor which of the dark of which he sat which i us was and upon a feeling of where eos the states an who of the dark and lifted upon the dull of at at his thus heard eos i heard lips as and of his countenance eos and these with a wild heart an did and the at her know the us and ethelred entering the door was nearly the ordinary madeline was was was we how a distribute the works for was my exceeding ordinary which was entering upon his face was a lips and who was the door and door was who it it in the sullen of the floor at which eos was us his were who ethelred upon it by his at of the chamber of the dark of the did he spoke of a visit at upon his sound he at her upon the spoke which he were of a her his spoke the

Chapter 3

Wanderers in that happy valley of the old old satyrs and the deep and dank tarn at my feet on the the now lifted did to the now lifted lifted at his lips struck as it was i was an mace was and ethelred ethelred was this was an mad law and it was an an foundation the door which now images of the the like the was nearly the even of the literary of the 8 of the spoke of her spoke of access of its lips of its days of and like her upon the of its spoke of her hear upon her her upon the works in a us her at before although had thus at the feet and thus her first the walls the hung of the days ethelred her ordinary of the ordinary ordinary now lifted lifted dull dull was struck the now he was death coffin to the now has has death death upon the visit of the coffin and lifted now lifted favourite of it it he it because to the us of which he sat down and us upon the feeling of the of the of the of the of the works at the now where i

Chapter 4

All with pearl and ruby glowing was the fair palace door of the chamber of the chamber and the he she gazed find upon and coffin and govern and distribute his now and upon his was it to as and a nature of the of the where there had of his friend he was a few struck upon the coffin he was the was reason did to head he has struck his spoke of a at of of of a at of of the visit at of a sat of it wild lips up by me might us of his her who upon his at his now her immediately fro upon his his immediately spoke of it her spoke of it was was her bounden into of his floor eos and were some burden his lofty and upon the hours the upon the of the which he was she to ethelred with the feeling of the of the of the of the was of the spoke in her spoke of the of the works in the works of the were s spoke of the heard at could was upon the once spoke of still visit her spoke of the was of the spoke of and hear donations eos and an who

Chapter 5

But evil things, in robes of sorrow states if an strange species of the phrase who and the deep and the simple landscape of the chamber of the heard the works was us upon the was of upon as his upon a of of wild links and although he hear a how of wild lofty of a dark of the work a were who at upon a immediately who he should dull upon the face and a now however it in a one of the the of the of the of the of the of the of and of and her spoke of its immediately lips and upon a lips of his in his her her works to gutenberg down her upon a her of the sound and his tattered now fro upon the tattered of the which he was to his the to the now did fell the electronic was ethelred now fell to his lips of his the now of his the death of the felt of the visit lifted of the heard the of the of the of the of the of a spoke of her spoke of heart lips and me upon a us her at at her dark and upon a distribution

### More on Natural language Processing and Language model
1. https://medium.com/@ageitgey/natural-language-processing-is-fun-9a0bff37854e
2. https://medium.com/phrasee/neural-text-generation-generating-text-using-conditional-language-models-a37b69c7cd4b
3. http://karpathy.github.io/2015/05/21/rnn-effectiveness/

**Music generates by RNN**
https://soundcloud.com/optometrist-prime/recurrence-music-written-by-a-recurrent-neural-network
